# Week 1 Assignment: ARIMA Time Series Forecasting

**Contents:**
- Data collection (Yahoo Finance)
- Preprocessing & stationarity testing
- Train/test split (80/20)
- ADF-based selection of `d` (0 or 1)
- Grid search (AIC) for best `(p, d, q)` on **training set only**
- Forecasting and correct index alignment
- Evaluation: MAE, MSE, RMSE
- Residual analysis

> This notebook is set up to avoid common pitfalls: no data leakage (models fit on training only), no double-differencing, and correct conversion to level forecasts when needed.


In [ ]:
# Install requirements if you don't have them (uncomment to run)
# !pip install yfinance statsmodels scikit-learn nbformat

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import yfinance as yf
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error

print('Libraries loaded')


## 1) Data Collection
We download historical stock closing prices from Yahoo Finance. Change the `TICKER`, `START`, and `END` as required.

For reproducibility in a classroom environment you can replace this with a local CSV file if internet access is restricted.


In [ ]:
# Parameters - change these if you want
TICKER = 'TSLA'   # e.g., 'TSLA', 'MSFT', 'GOOG'
START = '2023-01-01'
END = '2024-12-31'

# Download data
raw = yf.download(TICKER, start=START, end=END)
raw = raw.sort_index()
raw.head()


## 2) Preprocessing
- Use only the `Close` price for forecasting.
- Handle missing values by forward-fill.
- (Optional) Resample if you prefer weekly/monthly data.


In [ ]:
# Select closing price
ts = raw['Close'].copy()

# Check missing values
print('Missing values before:', ts.isna().sum())

# Forward-fill missing values
ts = ts.fillna(method='ffill')
print('Missing values after:', ts.isna().sum())

# Simple plot
plt.figure(figsize=(12,4))
plt.plot(ts)
plt.title(f'{TICKER} Close Price')
plt.ylabel('Price')
plt.show()


## 3) Stationarity Test (ADF) and choosing `d`
We use the Augmented Dickey-Fuller (ADF) test to check stationarity. If the series is non-stationary (p-value >= 0.05) we set `d=1`; otherwise `d=0`.


In [ ]:
# ADF test on full series (for info)
adf_full = adfuller(ts.dropna())
print('ADF statistic (full):', adf_full[0])
print('p-value (full):', adf_full[1])

# We'll split into train/test first and use train to decide d (preventing lookahead)

# Train-test split index
train_size = int(len(ts) * 0.8)
train = ts.iloc[:train_size]
test = ts.iloc[train_size:]

print('\nTrain length:', len(train), 'Test length:', len(test))

# ADF on train
adf_train = adfuller(train.dropna())
print('ADF statistic (train):', adf_train[0])
print('p-value (train):', adf_train[1])

# Decide d based on train ADF p-value
if adf_train[1] < 0.05:
    d_decision = 0
else:
    d_decision = 1

print('\nSelected d =', d_decision, '(0 => stationary, 1 => needs first differencing)')


### ACF & PACF (visual guide to choose p and q)
Plot ACF and PACF of the (possibly differenced) training series to get an idea of suitable `p` and `q` ranges.


In [ ]:
# Create series to analyze for ACF/PACF depending on d
if d_decision == 1:
    train_for_ac = train.diff().dropna()
else:
    train_for_ac = train.copy()

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plot_acf(train_for_ac, ax=plt.gca(), lags=30)
plt.title('ACF (train)')

plt.subplot(1,2,2)
plot_pacf(train_for_ac, ax=plt.gca(), lags=30, method='ywm')
plt.title('PACF (train)')
plt.tight_layout()
plt.show()


## 4) Grid Search on Training Set (AIC)
We perform a grid search **on the training set only**. `d` is fixed to the value chosen via the ADF test on the training set. We search over small p and q ranges to avoid overfitting and convergence issues.


In [ ]:
import itertools

p_range = range(0, 4)
q_range = range(0, 4)

best_aic = np.inf
best_order = None
best_model = None
results = []

for p, q in itertools.product(p_range, q_range):
    order = (p, d_decision, q)
    try:
        model = ARIMA(train, order=order)
        fit = model.fit()
        aic = fit.aic
        results.append((order, aic))
        if aic < best_aic:
            best_aic = aic
            best_order = order
            best_model = fit
        print(f"Tested ARIMA{order} - AIC: {aic:.2f}")
    except Exception as e:
        # print('Failed for', order, '->', e)
        continue

print('\nBest order:', best_order, 'with AIC =', best_aic)


### Best model summary
The best model from grid search is retained as `best_model` (fitted on the training set). We print a summary for inspection.


In [ ]:
if best_model is not None:
    print(best_model.summary())
else:
    print('No model was successfully fitted. Consider widening ranges or checking data.')


## 5) Forecasting
We forecast for the length of the test set. Since `best_model` was trained on the original train series and d was set inside the ARIMA call, the `forecast()` call will produce level forecasts (not differences). We'll ensure the forecast is converted into a pandas Series with the same index as `test`.


In [ ]:
# Forecast steps = len(test)
steps = len(test)

if best_model is None:
    raise RuntimeError('No fitted model available from grid search.')

forecast = best_model.forecast(steps=steps)
# Convert forecast into Series and align index
forecast = pd.Series(forecast, index=test.index)

# Plot train, test, forecast
plt.figure(figsize=(12,5))
plt.plot(train.index, train, label='Train')
plt.plot(test.index, test, label='Test')
plt.plot(forecast.index, forecast, label='Forecast')
plt.legend()
plt.title(f'ARIMA Forecast (order={best_order})')
plt.show()

# Show a small numeric preview
pd.DataFrame({'test': test, 'forecast': forecast}).head(10)


## 6) Evaluation
Compute MAE, MSE, and RMSE on the test set. These metrics quantify forecast accuracy.


In [ ]:
mae = mean_absolute_error(test, forecast)
mse = mean_squared_error(test, forecast)
rmse = np.sqrt(mse)

print(f'MAE: {mae:.4f}')
print(f'MSE: {mse:.4f}')
print(f'RMSE: {rmse:.4f}')

# Put results in a small table
pd.DataFrame({'MAE':[mae], 'MSE':[mse], 'RMSE':[rmse]})


## 7) Residual Analysis
Analyze residuals from the fitted model (on training data) and forecast errors on the test data.
- Residuals (train): should resemble white noise
- Forecast errors (test - forecast): check distribution and autocorrelation


In [ ]:
# Residuals from training fit
resid = best_model.resid

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(resid)
plt.title('Training residuals')

plt.subplot(1,2,2)
resid.plot(kind='kde')
plt.title('Residual density')
plt.tight_layout()
plt.show()

# Forecast errors on test
errors = test - forecast

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(errors)
plt.title('Forecast errors (test - forecast)')

plt.subplot(1,2,2)
errors.plot(kind='kde')
plt.title('Error density')
plt.tight_layout()
plt.show()

# ACF of errors - should be close to zero
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(errors.dropna(), lags=20)
plt.title('ACF of forecast errors')
plt.show()


## Conclusion & Next Steps
- The grid search finds the `(p,d,q)` that minimizes AIC **on the training set** to avoid data leakage.
- Forecasts were produced for the test set only and evaluated with MAE/MSE/RMSE.

**Possible improvements:**
- Use walk-forward validation for more robust evaluation.
- Consider SARIMA if seasonality exists.
- Explore LSTM or Prophet for non-linear patterns.

---
